In [ ]:
#***********************************************************************************
#                             IMPORT REQUIRED LIBRARIES 
#***********************************************************************************
import pandas as pd
import matplotlib as plt
import numpy as np 
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

In [ ]:
#***********************************************************************************
#                             IMPORT DATA FILES 
#***********************************************************************************

folderPath = "C:\\Users\\parker\\Dropbox\\Data Science Program\\Fall 2017\\DSBA6156 - Machine Learning\\Group Project - Yelp\\NC_SC_DataTables\\"
tar = os.chdir(folderPath)
print os.getcwd()

#df = pd.read_csv('elite_user_view.csv', low_memory=False, nrows=1000000)
df = pd.read_csv('user_review_stats.csv', low_memory=False)#, nrows=500000)

In [ ]:
#***********************************************************************************
#                             DROP UNWANTED FEATURES
#***********************************************************************************

df.head(5)
drop_col = ['user_name','review_count','yelping_since','user_id', 'average_stars','elite_user_flag','elite_years']

In [ ]:
#Set Index on user's ID
df = df.set_index('id')
df_elite = df['elite_user_flag']
df_user_name = df['user_name']

In [ ]:
#Drop features from dataframe & verify
df = df.drop(drop_col, axis=1 )
df.head(5)

In [ ]:
#***********************************************************************************
#                             CREATE NUMPY MATRIX FOR PLOTTING
#***********************************************************************************

col = ['useful','funny','cool','fans','compliment_hot',
       'compliment_more','compliment_profile','compliment_cute','compliment_list','compliment_note',
       'compliment_plain','compliment_cool','compliment_funny','compliment_writer','compliment_photos',
       'usr_acct_age_in_years','NbrReviews','NbrUsefulReviews','NbrFunnyReviews','NbrCoolReviews',
       'AvgStarsPerReview','NbrStarsTotal']

data = df.as_matrix(columns = col)
data[:5]

In [ ]:
#***********************************************************************************
#                             START KMEANS CLUSTERING EXPLORATION
#***********************************************************************************

########################################
k=3 #set number of clusters
########################################

clf = cluster.KMeans(n_clusters=k, init='k-means++', n_init=10)
clf.fit(df)

In [ ]:
#Get labels and centers

labels = clf.labels_
centroids = clf.cluster_centers_

labels[:50]


In [ ]:
#Plot the clusters with code borrowed from:
#http://fromdatawithlove.thegovans.us/2013/05/clustering-using-scikit-learn.html
#This code is reused for each cluster plot in this notebook

from matplotlib import pyplot

for i in range(k):
    # select only data observations with cluster label == i
    ds = data[np.where(labels==i)]
    # plot the data observations
    pyplot.plot(ds[:,0],ds[:,1],'o',label="Cluster: " + str(i+1))
    # plot the centroids
    lines = pyplot.plot(centroids[i,0],centroids[i,1],'kx')
    # make the centroid x's bigger
    pyplot.setp(lines,ms=15.0)
    pyplot.setp(lines,mew=2.0)
pyplot.legend(loc='upper left')
pyplot.show()

In [ ]:
#***********************************************************************************
#                             USE PCA TO REDUCE DIMENSIONS
#                Some code borrowed from inclass exercises and examples
#***********************************************************************************

from sklearn.decomposition import PCA
# load data
X = data[:,0:8]
Y = data[:,8]
# use three components 
pca = PCA(n_components=3)
fit = pca.fit(data)
# summarize components
print("Explained Variance: %s") % fit.explained_variance_ratio_
print(fit.components_)

In [ ]:
#apply the principle components to the data
data_pca = pca.fit_transform(data)

In [ ]:
#***********************************************************************************
#                             EXPLORE KMEANS ON PCA DATA & PLOT
#                Some code borrowed from inclass exercises and examples
#***********************************************************************************


clf = cluster.KMeans(n_clusters=k, init='k-means++', n_init=10)
clf.fit(data_pca)

labels = clf.labels_
centroids = clf.cluster_centers_

labels[:50]

for i in range(k):
    # select only data observations with cluster label == i
    ds = data_pca[np.where(labels==i)]
    # plot the data observations
    pyplot.plot(ds[:,0],ds[:,1],'o',label="Cluster: " + str(i+1))
    # plot the centroids
    lines = pyplot.plot(centroids[i,0],centroids[i,1],'kx')
    # make the centroid x's bigger
    pyplot.setp(lines,ms=15.0)
    pyplot.setp(lines,mew=2.0)
pyplot.legend(loc='upper left')
pyplot.show()

In [ ]:
#Create new data frame with just index and principle components
pca_data = pd.DataFrame(pca.fit_transform(df),columns=["pc1","pc2","pc3"])




In [ ]:
#***********************************************************************************
#                    EXPLORE THE CLUSTERS BY BRINGING IN FEATURES
#***********************************************************************************

#reset index to realign pca_data with user ids
new_index = pd.DataFrame(df)
new_index = new_index.reset_index()
print(type(new_index))
print(type(pca_data))

#merge pca data to original data frame
merge_back = pd.merge(new_index, pca_data, right_index=True, left_index=True)

#set merged data index as ID
merge_back = merge_back.set_index('id')
#print(merge_back.info())

#recreate principle components with new index (user id) for clustering
pca_allusers_cluster = merge_back[["pc1","pc2","pc3"]]#,"pc4"]]

#clean up naming
df_pca = pca_allusers_cluster

In [ ]:
#***********************************************************************************
#          RUN KMEANS AGAIN TO OBSERVE RESULTS WITH NEW DATA FRAME & INDEX
#                            Identify Clusters after kmeans
#***********************************************************************************


clf = cluster.KMeans(n_clusters=k, init='k-means++', n_init=10)
clf.fit(df_pca)

data_pca = df_pca.as_matrix()

labels = clf.labels_
centroids = clf.cluster_centers_

labels[:50]

for i in range(k):
    # select only data observations with cluster label == i
    ds = data_pca[np.where(labels==i)]
    # plot the data observations
    pyplot.plot(ds[:,0],ds[:,1],'o',label="Cluster: " + str(i+1))
    # plot the centroids
    lines = pyplot.plot(centroids[i,0],centroids[i,1],'kx')
    # make the centroid x's bigger
    pyplot.setp(lines,ms=15.0)
    pyplot.setp(lines,mew=2.0)
pyplot.legend(loc='upper left')
pyplot.show()

#Shows consistant results -- proceed

In [ ]:
#Get cluster Numbers
x = clf.fit_predict(df_pca)

In [ ]:
#update data frame with cluster number
df_pca['Cluster'] = x

In [ ]:
#spy data to ensure cluster was updated
df_pca.head(5)

In [ ]:
#***********************************************************************************
#          EXPLORE THE CLUSTERS AND REIDENTIFY ELITE USERS
#***********************************************************************************

#check to make sure elite user data was preserved
df_elite.head(5)

In [ ]:
#Checking pandas type
print(type(df_pca))
print(type(df_elite))

#df_elite is still series - convert to frame and merge elite user flag back into data frame
clustered_users = pd.merge(df_pca, df_elite.to_frame(), right_index=True, left_index=True)

clustered_users.head(5)


In [ ]:
#Group by cluster and elite user to identify the predictor cluster
#!!!!!!! ---- CHECK CLUSTER NUMBER --------!!!!!!!!!!
#ImportantComment1
#Cluster likely changes each attempt at kmeans and we need to update the predict_cluster

clustered_users.groupby(['Cluster','elite_user_flag']).size()

In [ ]:
#***********************************************************************************
#                             ADD DISTANCE FROM CLUSTER CENTER
#                             TO THE DATAFRAME
#***********************************************************************************

clusters_df = clustered_users["Cluster"]
#print(clusters_df.head)

In [ ]:
#Compute clustering and transform X to cluster-distance space
#Returns:  X_new : array, shape [n_samples, k]

y = clf.fit_transform(df_pca)
#y
print(y.shape)

In [ ]:
#set distances from centers & create new column in data frame
distances = list()

for index, cluster in enumerate(clusters_df):
        distances.append(min(y[index]))

clustered_users["cluster_distances"] = distances
clustered_users.head(20)

In [ ]:
#***********************************************************************************
#                             MERGE DESCRIPTOR FEATURES BACK IN AFTER PCA
#***********************************************************************************

df = df.merge(clustered_users,right_index=True,left_index=True)
df.head(10)

In [ ]:
#***********************************************************************************
#                             IDENTIFY AND EXPLORE THE CLUSTERS 
#***********************************************************************************

Cluster1 = df.loc[df["Cluster"] == 0]
Cluster2 = df.loc[df['Cluster'] == 1]
Cluster3 = df.loc[df['Cluster'] == 2]
#Cluster4 = df.loc[df['Cluster'] == 3]
#Cluster5 = df.loc[df['Cluster'] == 4]
#Cluster6 = df.loc[df["Cluster"] == 5]
#Cluster7 = df.loc[df['Cluster'] == 6]
#Cluster8 = df.loc[df['Cluster'] == 7]
#Cluster9 = df.loc[df['Cluster'] == 8]
#Cluster10 = df.loc[df['Cluster'] == 9]



In [ ]:
Cluster1.describe(include = 'all')

In [ ]:
Cluster2.describe(include = 'all')

In [ ]:
Cluster3.describe(include = 'all')

In [ ]:
#Use cluster identified at the position of ImportantComment1 as the predictor 
#since the number of elite users is much higher than non-elite users 

#Plot elite users vs. non elite users in predictor cluster
predict_cluster = Cluster3 #SET THIS AS CLUSTER IDENTIFIED IN ImportantComment1

#_________________________________________________________________________________
plt.scatter(predict_cluster['elite_user_flag'], predict_cluster["cluster_distances"])
plt.legend()
plt.show()

In [ ]:
#**************************************************************************************
#                              RANK THE NON_ELITE USERS BY DISTANCE FROM CENTER 
#                                 TO FIND NEXT SET OF ELITE USERS
#  ***************************************************************************************

# reference: https://stackoverflow.com/questions/17141558/how-to-sort-a-dataframe-in-python-pandas-by-two-or-more-columns
#sort distances
predict_cluster = predict_cluster.sort_values(["cluster_distances"], ascending=True)
#get top 20 non elite users as next generation of elite users
top_20_next_elites = predict_cluster[predict_cluster['elite_user_flag']==0].head(20)
#Merge First Names back for ease of human identification
top_20_next_elites = top_20_next_elites.merge(df_user_name.to_frame(),right_index=True,left_index=True)

In [ ]:
#**************************************************************************************
#                           EXPLORE THE NEXT GENERATION OF ELITE USERS 
#                                        (LIMIT 20 USERS)
#  ***************************************************************************************
print(top_20_next_elites.describe(include='all'))
top_20_next_elites

In [ ]:
#Get top 20 names
top_20_next_elites[['cluster_distances','user_name']]